In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
import os, sys 
sys.path.append(os.path.expanduser("~/git/butools/Python"))
import butools 
from butools.ph import *

In [3]:
sys.path.append(os.path.abspath(".."))
from utils import compute_first_n_moments

In [4]:
# Test 
a, A = APHFrom2Moments([1, 2])
MomentsFromPH(ml.matrix(a), ml.matrix(A))

[1.0, 2.0, 6.0]

In [5]:
# Test 
a, A = RandomPH(order=2)
MomentsFromPH(ml.matrix(a), ml.matrix(A))

[0.9999999999999999, 2.1103189377260576, 6.7550119739413725]

In [6]:
def sample_coxian(degree, max_rate):
    lambdas = np.random.rand(degree) * max_rate 
    ps = np.random.rand(degree - 1) 
    A = np.diag(-lambdas) + np.diag(lambdas[:degree-1] * ps, k=1)
    alpha = np.eye(degree)[[0]]
    return alpha, A

In [7]:
# Example 
a, A = sample_coxian(degree=3, max_rate=1)
MomentsFromPH(ml.matrix(a), ml.matrix(A))

[156.91738854641022,
 48784.25725555411,
 22747760.92823944,
 14142817454.63645,
 10991152285461.98]

In [8]:
compute_first_n_moments(a, A, n=5)

array([1.56917389e+02, 4.87842573e+04, 2.27477609e+07, 1.41428175e+10,
       1.09911523e+13])

In [12]:
A2 = A * compute_first_n_moments(a, A, n=5)[0]
compute_first_n_moments(a, A2, n=5)

array([ 1.        ,  1.85566694,  5.02136724, 17.90113736, 79.31650392])

### Stats of coxian samples 

In [9]:
degree = 5
max_rate = 5
moments = []

for _ in range(10000):
    a, A = sample_coxian(degree=degree, max_rate=max_rate)
    m = MomentsFromPH(a, A)
    moments.append(m)

In [10]:
moments = pd.DataFrame(moments)
t = moments.describe()
t.columns = [f"Moment-{m+1}" for m in t.columns]
t

,Moment-1,Moment-2,Moment-3,Moment-4,Moment-5,Moment-6,Moment-7,Moment-8,Moment-9
count,10000.000000,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,1.784938,8.668176e+02,3.762312e+06,2.645799e+10,2.391689e+14,2.606984e+18,3.318312e+22,4.827996e+26,7.902879e+30
std,20.742850,6.640297e+04,3.610375e+08,2.626238e+12,2.388262e+16,2.606233e+20,3.318115e+24,4.827936e+28,7.902858e+32
min,0.200061,8.004847e-02,4.804363e-02,3.844655e-02,3.845819e-02,4.616381e-02,6.464892e-02,1.034696e-01,1.863017e-01
25%,0.264598,1.400244e-01,1.111506e-01,1.176409e-01,1.556379e-01,2.470889e-01,4.576549e-01,9.687572e-01,2.306982e+00
50%,0.400704,3.211273e-01,3.860309e-01,6.187365e-01,1.239651e+00,2.980398e+00,8.359802e+00,2.679845e+01,9.664422e+01
75%,0.825560,1.363100e+00,3.375965e+00,1.114825e+01,4.601778e+01,2.279428e+02,1.317264e+03,8.699848e+03,6.464025e+04
max,1818.779821,6.615920e+06,3.609871e+10,2.626224e+14,2.388262e+18,2.606233e+22,3.318115e+26,4.827936e+30,7.902858e+34
